# Рекуррентные сети пишут тексты
__Суммарное количество баллов: 12__

__Решение отправлять на `ml.course.practice@gmail.com`__

__Тема письма: `[ML][MS][HW06] <ФИ>`, где вместо `<ФИ>` указаны фамилия и имя__

В этом задании вам предстоит познакомиться с объединением Deep Learning и NLP. Для начала предстоит построить векторное пространство для словоря, а затем применить его для предсказания следующих слов в тексте.

In [0]:
import torch
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import random
import json
import copy
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
from sklearn.neighbors import KDTree
import time
from collections import defaultdict
from torch.autograd import Variable
import itertools

In [0]:
tokenized = json.load(open("opencorp.json", "r", encoding="UTF-8"))

### Задание 1 (3 балла)
Чтобы обучить нейронную сеть, нам нужен датасет. В данном задании предлагается использовать данные, полученные из корпуса текстов OpenCorpora. Более того, датасет нужно представить в удобном виде. Поскольку мы хотим обучать эмбеддинги на парах `(token_center, token_context)`, а также иметь возможность делать `negative sampling`, датасет должен уметь выдавать соответствующие пары, а так же `negative sampling`-токены. 

Кроме того, мы бы не хотели строить эмбеддинги для очень редких слов, поэтому в словарь и в пары должны входить только слова, которые встречаются более `count_threshold` раз, а остальные должны быть заменены на специальный токен `"<UNKNOWN>"`. Последовательность должна начинаться с токена `"<START>"` и заканчиваться токеном `"<END>"`.

#### Методы
`__init__` - принимает на вход список последовтельностей токенов, преобразуя в соответствии с описанными выше критериями. При инициализации списка токенов нужно учитывать, что с вероятностью $1 - (\sqrt{\frac{0.001}{f(t)}} + 1) \cdot \frac{f(t)}{0.001}$ ($f(t)$ - частота слова в корпусе) мы "выкидываем" слово из текста, не добавляя никакие пары токенов с его участием в список. Это нужно для того, чтобы мы не переобучались на часто встречаемые слова. Также в `self.voc` должен записать актуальный словарь токенов.

`__len__` - возвращает количество пар `(token_center, token_context)`

`__getitem__` - принимает на вход индекс `i`, соответствующий паре `(t_i, c_i)`. Возвращает пару тензоров `(t_i, [c_i] + negatives)`, где `negatives` - список негативных токенов.

`negative_sampling` - осуществляет взвешенное негативное сэмплирование. Вес токена определяется как $\frac{(count(t))^{0.75}}{\sum (count(t))^{0.75}}$, т.е. в negative samples частые слова попадают чаще, чем другие.

In [0]:
class TokenDataset(Dataset):
    def __init__(self, tokenized_sources, window=3, count_threshold=5, negative_sampling=5):
        self.token_pairs = []
        self.neg_samp = negative_sampling
        self.counts = {"<START>": len(tokenized_sources), "<END>": len(tokenized_sources), "<UNKNOWN>":0}
        
        total_words_count = 2 * len(tokenized_sources)
        for sent in tokenized_sources:
            total_words_count += len(sent)
            for word in sent:
                if word in self.counts:
                    self.counts[word] += 1
                else:
                    self.counts[word] = 1
        
        rem_words = []
        for word in self.counts:
            if self.counts[word] < count_threshold:
                if word not in ["<UNKNOWN>", "<START>", "<END>"]:
                    self.counts["<UNKNOWN>"] += self.counts[word]
                    rem_words.append(word)
        [self.counts.pop(word) for word in rem_words]

        to_rem = copy.deepcopy(tokenized_sources)
        for sent in to_rem:
            for i  in range(len(sent)):
                if sent[i] in self.counts:
                    word = sent[i]
                else:
                    word = "<UNKNOWN>"
                    sent[i] = "<UNKNOWN>"

                if word not in ["<START>", "<END>"]:
                    f = self.counts[word] / total_words_count * 1000
                    f = 1 / f
                    if np.random.rand() < 1 - np.sqrt(f) - f:
                        sent[i] = "<DELETED>"
        
        for sent in to_rem:
            ns = ["<START>"] + sent + ["<END>"]
            for i, word in enumerate(ns):
                if word in self.counts:
                    word_to_add = word

                    if word == "<DELETED>":
                        continue

                    if word not in self.counts:
                        word_to_add = "<UNKNOWN>"

                    for j in range(-window, window+1):
                        pos = i + j
                        if pos >= 0 and pos < len(ns) and pos != i and ns[pos] != "<DELETED>":
                            self.token_pairs.append((word_to_add, ns[pos]))

        self.voc = np.array(list(self.counts.keys()))
        self.token2id = dict([(token, i) for i, token in enumerate(self.voc)])
        self.lc = len(self.voc)
        den = 0
        for word in self.counts:
            den += pow(self.counts[word], 0.75)

        self.prob = [pow(self.counts[word], 0.75) / den for word in self.voc]
                    
    def negative_sampling(self):
        ind = np.random.choice(np.arange(self.lc), p=self.prob, size=self.neg_samp)
        return list(ind)
        
    def __getitem__(self, index):
        ns = self.negative_sampling();
        t, c = self.token_pairs[index]

        t = self.token2id[t]
        c = self.token2id[c]
        

        return torch.tensor([t]), torch.tensor([c] + ns, dtype=torch.long)

    def __len__(self):
        return len(self.token_pairs)



### Задание 2 (4 балла)
Теперь реализуем непосредственно SkipGram. Для этого нам потребуются `torch.autograd.Variables` чтобы сделать эмбеддинги обучаемыми. Также сразу реализуем интерфейс, которым будем пользоваться для применения эмбеддингов в следующих задачах.

#### Методы SkipGram
`__init__` - принимает на вход словарь и размерность пространств эмбеддингов. Инициализирует эмбеддинги для центрального и контекстного слов.

`get_variables` - возвращает лист из всех `torch.autograd.Variables`. Необходимо, чтобы инициализировать оптимизатор.

`predict_proba(center_tokens, context_tokens)` - принимает на вход список центральных токенов и список списков токенов из предполагаемого контекста. Для каждого центрального токена и соответствующего ему списка контекстных токенов должен вернуть скалярное произведение центрального и контекстуального эмбеддингов.

#### Методы Embedding
`__init__` - принимает на вход обученный SkipGram

`embed` - возвращает эмбеддинги для всех элементов списка

`reconstruct` - для всех элементов списка возвращает наиболее подходящий токен. Не возвращает `"<UNKNOWN>"`

`n_closest` - возвращает `n` ближайших токенов для каждого элемента списка. Не возвращает `"<UNKNOWN>"`

In [0]:
class SkipGram:
    def __init__(self, voc, latent_size):
        self.id2token = list(voc)
        self.token2id = dict([(token, i) for i, token in enumerate(self.id2token)])
        self.v = Variable(torch.randn(len(voc) ,latent_size).float().cuda(), requires_grad=True)
        self.u = Variable(torch.randn(latent_size, len(voc)).float().cuda(), requires_grad=True)
        self.ls = latent_size
        
    def get_variables(self):
        return [self.v, self.u]
    
    def predict_proba(self, center_ids, context_ids):
        bs = context_ids.shape[0]
        ns = context_ids.shape[1]  
        ls = self.ls
        res = torch.bmm(self.v[center_ids], self.u[:,context_ids].transpose(1, 0))
        return res.reshape(bs, ns).cuda()
            
class Embedding:
    def __init__(self, skip_gram):
        self.skip_gram = skip_gram
        self.tree = KDTree(skip_gram.v.detach().numpy())
    
    def embed(self, tokens):
        res = torch.zeros(len(tokens) , self.skip_gram.ls)
        for i, token in enumerate(tokens):
            if token in self.skip_gram.token2id:
                res[i] = self.skip_gram.v[self.skip_gram.token2id[token]]
            else:
                res[i] = self.skip_gram.v[self.skip_gram.token2id["<UNKNOWN>"]]
        return res.detach().numpy()
    
    def reconstruct(self, embeddings):
        _, inds = self.tree.query(embeddings, 2)
        res = []
        for ind in inds:
            if not self.skip_gram.id2token[ind[0]] == "<UNKNOWN>":
                res.append(self.skip_gram.id2token[ind[0]])
            else:
                res.append(self.skip_gram.id2token[ind[1]])
        return res

    def n_closest(self, embeddings, n=5):
        res = [[] for _ in range(len(embeddings))]
        _, inds = self.tree.query(embeddings, k=n+1)
        for i, _ in enumerate(embeddings):
            for ind in inds[i]:
                if self.skip_gram.id2token[ind] != "<UNKNOWN>" and len(res[i]) < n:
                    res[i].append(self.skip_gram.id2token[ind])
        return res

In [0]:
class MyBatch:
    def __init__(self, dataset, batch_size=4096, shuffle=False):
        self.ds = dataset
        self.bs = batch_size
        self.sf = shuffle
    
    def batches(self):
        bs = self.bs
        ind = np.arange(len(self.ds))
        if self.sf:
            np.random.shuffle(ind)
        res = []
        for i in range(int(np.ceil(len(self.ds) / bs))):
            if i % 100 == 99:
                print(i)
            cur = []
            cur.append(torch.Tensor(ind[bs*i:(i+1)*bs]).reshape(bs, 1))
            sec = None
            for j in ind[bs*i:(i+1)*bs]:
                if sec is None:
                    sec = self.ds[j][1]
                else:
                    torch.cat((sec, self.ds[j][1]), 0)
            cur.append(sec)
            res.append(cur)
        return res
                
        

    

In [0]:
dataset = TokenDataset(tokenized, count_threshold=20, window=3, negative_sampling=3)

In [0]:
BATCH_SIZE = 4096
EPOCHS = 5

print("Creating dataset...")
dataset = TokenDataset(tokenized, count_threshold=20, window=3, negative_sampling=2)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
print("Creating skipgram...")
skipgram = SkipGram(dataset.voc, 12)
optim = torch.optim.Adam(skipgram.get_variables(), lr=1e-3)
y = torch.tensor([0] * (BATCH_SIZE + 1), dtype=torch.long).cuda()
losses = []

for i in range(EPOCHS):
    avg_loss = 0
    steps = 0
    ct = time.time()
    for batch in tqdm(dataloader):
        probs = skipgram.predict_proba(batch[0], batch[1])
        loss = F.cross_entropy(probs, y[:len(probs)])
        optim.zero_grad()
        loss.backward()
        optim.step()
        avg_loss += loss.item()
        steps += 1
        print(steps)
    losses.append(avg_loss/steps)
    print("Loss:", avg_loss/steps)

plt.figure(figsize=(12, 8))
plt.plot(list(range(EPOCHS)), losses)
plt.tight_layout()
plt.show()

In [37]:
print("Creating dataset...")
dataset = TokenDataset(tokenized, count_threshold=20, window=3, negative_sampling=2)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

Creating dataset...


In [0]:
ct = time.time()
for batch in tqdm(dataloader):
    print(time.time()-ct)
    ct = time.time()

In [0]:
tt = t.batches()

In [0]:
king = embedder.embed(["король"])[0]
cat = embedder.embed(["кошка"])[0]
owl = embedder.embed(["сыч"])[0]
give = embedder.embed(["дать"])[0]
me = embedder.embed(["ты"])[0]
you = embedder.embed(["я"])[0]
print(embedder.n_closest([king], 10)[0])
print(embedder.n_closest([cat], 10)[0])
print(embedder.n_closest([owl], 10)[0])
print(embedder.n_closest([give], 10)[0])
print(embedder.n_closest([me], 10)[0])
print(embedder.n_closest([you], 10)[0])

['король', 'итогам', 'было', 'о', 'то', 'общества', 'направлено', 'известных', 'ну', 'также']
['из', 'что', '-', '…', 'очень', 'которых', 'его', 'такой', 'ли', '10']
['из', 'что', '-', '…', 'очень', 'которых', 'его', 'такой', 'ли', '10']
['из', 'что', '-', '…', 'очень', 'которых', 'его', 'такой', 'ли', '10']
['ты', ':', 'же', 'себе', 'за', 'ее', 'понятия', 'их', 'в', 'стране']
['я', 'было', 'не', 'эти', 'по', 'от', 'всего', 'является', '–', 'могут']


### Задание 3 (2 балла)
Теперь будем учиться восстанавливать слова в тексте. Для этого нам потребуется также определить датасет последовательностей фиксированной длинны.

#### Методы
`__init__` - принимает на вход `embedder` (обученный SkipGram) и список токенизированных последоватлеьностей `tokenized`.

`__getitem__` - возвращает случайную закодированную при помощи SkipGram подпоследовательность длины `seq_len` одной из исходных последовательностей, сдвинутую на один токен подпоследовательность (т.е. следующие слова в тексте) и маску, которая отражает то, является ли токен неизвестным (`"<UNKNOWN"`).

`__len__` - равна количеству последовательностей.

In [0]:
class TokenSeqDataset(Dataset):
    def __init__(self, embedder, tokenized, seq_len=32):
        self.seq_len = seq_len
        self.tokenized = tokenized
        self.embedder = embedder
            
    def __len__(self):
        return len(tokenized)

    def __getitem__(self, index):
        top = max(len(self.tokenized[index]) - self.seq_len, 0)
            
        ind = np.random.randint(0, 1 + top)
        
        sent = copy.deepcopy(self.tokenized[index])
        sent = sent + ["<END>"] * (1 + max(0, self.seq_len-len(sent)))
        seq = sent[ind: ind + self.seq_len + 1]
        mask = [token in self.embedder.skip_gram.token2id
                for token in seq[1:]]
        mask = torch.Tensor(np.array(mask))
        resseq = [self.embedder.embed([word])[0] for word in seq]
        resseq = torch.Tensor(np.array(resseq))
        a1 = copy.deepcopy(resseq[:-1])
        a2 = copy.deepcopy(resseq[1:])
        return a1, a2, mask

### Задание 4 (2 балла)
Теперь обучим рекуррентную сеть, которая будет предсказывать следующее слово в тексте. Модель будет состоять из трех блоков: `input` (отвечает за предоброботку эмбеддинга), `rnn` (рекуррентная часть), `output` (отвечает за постобработку выхода).

#### Методы
`predict_sequential` - возвращает последовательность предсказаний для батча последовательностей

`get_next` - предсказывает следующее слово

`reset` - обнуляет внутреннее состояние сети

In [0]:
class TextRNN:
    def __init__(self, latent_space=150, hidden_layer=512):
        self.input = torch.nn.Sequential(
            torch.nn.Linear(latent_space, hidden_layer),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_layer, hidden_layer),
            torch.nn.ReLU()
        )
        self.rnn = torch.nn.LSTM(hidden_layer, hidden_layer)
        self.output = torch.nn.Linear(hidden_layer, latent_space)
        self.input.cuda()
        self.rnn.cuda()
        self.output.cuda()
        self.hidden = None
    
    def predict_sequential(self, sequences):
        x, _ = self.rnn(self.input(sequences))
        return self.output(x)
    
    def parameters(self):
        return list(self.input.parameters()) + list(self.rnn.parameters()) + list(self.output.parameters())
    
    def get_next(self, batch):
        x, self.hidden = self.rnn(self.input(sequences).unsqueeze(1))
        return self.output(x)
    
    def reset(self):
        self.hidden = None

In [0]:
BATCH_SIZE = 64
EPOCHS = 200

dataset = TokenSeqDataset(embedder, tokenized)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
rnn = TextRNN(24)
optim = torch.optim.Adam(rnn.parameters(), lr=1e-3)
losses = []
top1accs = []
top5accs = []
for i in range(EPOCHS):
    avg_loss = 0
    top1acc = 0
    top5acc = 0
    steps = 0
    acc_steps = 0
    for x, y_true, loss_mask in tqdm(dataloader):
        y_pred = rnn.predict_sequential(x.cuda())
        loss = (((y_true.cuda() - y_pred)**2).mean(dim=-1) * loss_mask.cuda()).mean()
        optim.zero_grad()
        loss.backward()
        optim.step()
        avg_loss += loss.item()
        steps += 1
        if steps % 100 == 0:
            acc_steps += 1
            word_pred = embedder.n_closest(y_pred.detach().view(-1, 24)[:200].cpu().numpy())
            word_true = embedder.reconstruct(y_true.detach().view(-1, 24)[:200].cpu().numpy())

            unknown_mask = loss_mask.view(-1).cpu().numpy()
            t1a = 0
            t5a = 0
            for true, pred, is_unknown in zip(word_true, word_pred, unknown_mask):
                if is_unknown:
                    continue
                if true == pred[0]:
                    t1a += 1
                if true in pred:
                    t5a += 1
            top1acc += t1a / len(word_pred)
            top5acc += t5a / len(word_pred)
    losses.append(avg_loss/steps)
    top1accs.append(top1acc/acc_steps)
    top5accs.append(top5acc/acc_steps)
    print("Loss:", avg_loss/steps)
    print("Top-1 accuracy:", top1acc/acc_steps)
    print("Top-5 accuracy:", top5acc/acc_steps)

plt.figure(figsize=(12, 8))
plt.plot(list(range(EPOCHS)), losses)
plt.tight_layout()
plt.show()
plt.figure(figsize=(12, 8))
plt.plot(list(range(100)), top1accs, label="Top-1")
plt.plot(list(range(100)), top5accs, label="Top-5")
plt.legend()
plt.tight_layout()
plt.show()

Loss: 0.09322435809825345
Top-1 accuracy: 0.0008333333333333334
Top-5 accuracy: 0.07916666666666666


Loss: 0.08952413314947258
Top-1 accuracy: 0.0008333333333333334
Top-5 accuracy: 0.012499999999999999


Loss: 0.08874099743722681
Top-1 accuracy: 0.0008333333333333334
Top-5 accuracy: 0.016666666666666666


Loss: 0.08851615568480618
Top-1 accuracy: 0.0016666666666666668
Top-5 accuracy: 0.010833333333333334


Loss: 0.08833593239023836
Top-1 accuracy: 0.0008333333333333334
Top-5 accuracy: 0.012499999999999999


Loss: 0.08838290732935615
Top-1 accuracy: 0.0025
Top-5 accuracy: 0.013333333333333334


Loss: 0.0879429124567992
Top-1 accuracy: 0.0008333333333333334
Top-5 accuracy: 0.015000000000000001


Loss: 0.08807886863418196
Top-1 accuracy: 0.0008333333333333334
Top-5 accuracy: 0.012500000000000002


KeyboardInterrupt: ignored

### Задание 5 (1 балл)
Отлично, осталось только научитсья итеративно продолжать последовательность. Давайте попробуем научиться это делать.

#### Методы
`continue_sequence` - возвращает завершенную последовательность. Входная последовательность может быть пустой, поэтому в начало нужно добавить токен `"<START>"`. Закончить построение последовательности нужно после получения токена `"<END>"` или после получения `max_len` новых слов.

In [0]:
class SequenceCompleter:
    def __init__(self, rnn, embedder, max_len=128):
        self.rnn = rnn
        self.embedder = embeder
        self.max_len = max_len
        
    def continue_sequence(self, sequence):
        t_sequence = ["<START>"] + sequence
        embedding = self.embedder.embed(t_sequence)
        self.rnn.reset()
        with torch.no_grad():
            for e in embedding:
                x = self.rnn.get_next(torch.tensor([e], dtype=torch.float).cuda())
            rec = self.embeder.reconstruct(x)
            continued_sequence = []
            ctn = 0
            while rec[0] != "<END>" and ctn < self.max_len:
                continued_sequence.append(rec[0])
                e = self.embedder.embed(rec)
                x = self.rnn.get_next(torch.tensor(e, dtype=torch.float).cuda())
                rec = self.embeder.reconstruct(x)
        return sequence + continued_sequence

In [0]:
seq_completer = SequenceCompleter(rnn, embedder)
print(seq_completer.continue_sequence(["учеба", "в", "магистратуре", "-", "это"]))
print(seq_completer.continue_sequence(["работает", "ли", "наша", "простая", "модель", "?"]))
print(seq_completer.continue_sequence(["я", "точно", "знаю"]))
print(seq_completer.continue_sequence(["машина", "времени"]))
print(seq_completer.continue_sequence(["сегодня"]))
print(seq_completer.continue_sequence([]))

In [0]:
class CharLSTM:
    def __init__(self, symbols):
        self.hidden = None
        self.input = torch.nn.Linear(len(symbols), 128)
        self.lstm = torch.nn.LSTM(128, 128)
        self.output = torch.nn.Sequential(torch.nn.Linear(128, 128), torch.nn.ReLU(), torch.nn.Linear(128, len(symbols)))
    
    def loss(self, batch):
        pass